In [ ]:
import read_data as rd
import pandas as pd
import numpy as np
import csv
import sklearn
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import visualizations as viz
import matplotlib.cm as cm
import itertools
%matplotlib inline

# Analysis Functions

These are used throughout the rest of the notebook. 

In [ ]:
def get_prev_nday(df, target, n=1, mean_window=7):
    min_yr = df.Date.dt.year.min()
    max_yr = df.Date.dt.year.max()
    yr_range = range(min_yr, max_yr+1)
    
    prev_df = pd.concat([df[df.Date.dt.year == year].groupby('Beach')\
                         .apply(lambda x: x.set_index('Date').resample('D').asfreq())\
                         .drop('Beach', axis=1).reset_index()
                         for year in yr_range])
    
    if isinstance(target, str):
        target = [target, ]
    
    prev_target = [str(n)+ '_prev_' + key for key in target]
    prev_df[prev_target] = prev_df.groupby(['Beach', prev_df.Date.dt.year]).shift()[target]
    prev_df.drop(target, axis=1, inplace=True)
    
    means = prev_df.groupby(['Beach', prev_df.Date.dt.year])[prev_target]\
                   .transform(lambda x: x.rolling(min_periods=1, center=False, window=mean_window).mean())
    
    prev_df[prev_target] = prev_df[prev_target].fillna(means)
    
    for yr in yr_range:
        means = prev_df[prev_df.Date.dt.year != yr].mean()
        prev_df.loc[prev_df.Date.dt.year == yr, prev_target] =\
        prev_df.loc[prev_df.Date.dt.year == yr, prev_target].fillna(means)
        
    
    return prev_df  

def run_model(df, model, predictors, target='Escherichia.coli', threshold=235, yrs=range(2006,2015)):
    
    model_df = df[['Date', target] + predictors].copy().dropna()
    
    for predictor in predictors:
        if model_df[predictor].dtype == 'object':
            model_df[predictor] = pd.Categorical(model_df[predictor]).codes
    
    fits = {}
    for yr in yrs:
        model = sklearn.base.clone(model) 
        train = model_df[model_df.Date.dt.year != yr][predictors]
        classes = model_df[model_df.Date.dt.year != yr][target] > threshold
        
        fits[yr] = model.fit(train, classes)
        
    return fits, model_df
        
def calc_metrics(model_df, predictors, fits, metrics=[], 
                 target='Escherichia.coli', threshold=235, gen_avg=False):
    
    calced_metrics = {}
    avg_calced_metrics = {}
    for metric in metrics:
        yr_scores = {}
        
        for yr, fit in fits.items():
            test = model_df[model_df.Date.dt.year == yr][predictors]
            true_classes = model_df[model_df.Date.dt.year == yr][target] > threshold
            
            if 'roc' in metric:
                preds = fit.predict_proba(test)[:,1]
            else:
                preds = fit.predict(test)
        
            yr_scores[yr] = getattr(sklearn.metrics, metric)(true_classes, preds)
        
        calced_metrics[metric] = yr_scores
        if gen_avg:
            try:
                avg_calced_metrics[metric] = np.array(list(calced_metrics[metric].values()))\
                                               .mean(axis=0)
            except ValueError: 
                print("Can't average %s" %metric)
                
    return (calced_metrics, avg_calced_metrics) if gen_avg else calced_metrics

def plot_roc(roc_dict, drek_df, roc_aucs=None):
    colors = itertools.cycle(sns.color_palette('colorblind', 6))
    
    for yr, arrays in metrics['roc_curve'].items():
        if roc_aucs is not None:
            label = str(yr) + " - AUC: {:0.4f}".format(roc_aucs[yr])
        else:
            label = yr
        fpr = arrays[0]
        tpr = arrays[1]
        if yr <= 2010:
            plt.plot(fpr, tpr, label=label, color=next(colors))
        else:
            plt.plot(fpr, tpr, '--', label=label, color=next(colors))
    
    plt.plot([0, 1], [0, 1], 'r:')
    fpr, tpr, _ = sklearn.metrics.roc_curve(drek_df['Escherichia.coli'] > 235, 
                                            drek_df['Drek_Prediction'])
    
    if roc_aucs is not None:
        drek_auc = sklearn.metrics.roc_auc_score(drek_df['Escherichia.coli'] > 235, 
                                                 drek_df['Drek_Prediction'])
        label = 'Drek - AUC: {:0.4f}'.format(drek_auc)
    else:
        label = 'Drek'
        
    plt.plot(fpr, tpr, 'k--', lw=2, label=label)

    plt.legend(loc='lower right', fontsize=12)
    plt.xlabel('False Positive Rate', fontsize=12)
    plt.ylabel('True Positive Rate', fontsize=12)
    plt.show()

def get_feat_df(fits, predictors):
    vals = [[yr, feat, imp] for yr, fit in fits.items() 
            for feat, imp in zip(predictors, fit.feature_importances_)] 
    
    return pd.DataFrame(vals, columns = ['Year', 'Feature', 'Importance'])

## Data Prep

In [ ]:
clean_df = rd.read_data(read_drek=True, read_holiday=False, read_weather_station=False,
                        read_water_sensor=False, read_daily_forecast=False, read_hourly_forecast=False,
                        group_beaches=False
                       )

In [ ]:
drek_df = clean_df[['Drek_Prediction', 'Escherichia.coli']].copy().dropna()

In [ ]:
df = clean_df.copy()
df.drop(['Reading.1', 'Reading.2', 'Units', 'Sample.Collection.Time', 
         'Date', 'Laboratory.ID', 'Weekday', 'Beach', 'Year',
         'Drek_Reading', 'Drek_Prediction', 'Drek_Worst_Swim_Status'], axis=1, inplace=True)
df = df[~df['Escherichia.coli'].isnull()]
df.rename(columns={'Full_date': 'Date', 'Client.ID': 'Beach'}, inplace=True)

cleannames_file = '../data/ChicagoParkDistrict/raw/Standard 18 hr Testing/cleanbeachnames.csv'
with open(cleannames_file) as f:
    next(f)
    cleannames_dict = {line[1]: line[3] for line in csv.reader(f)}
    
df.Beach.replace(cleannames_dict, inplace=True)

df = df[~df.duplicated(subset=['Date', 'Beach'])]

#we're just looking at years 2006-2014 for now
df = df[df.Date.dt.year < 2015]

### df for lat & lon for each beach

In [ ]:
beach_locs = pd.read_csv('../data/ExternalData/Beach_Locations.csv')
beach_locs.Beach.replace(cleannames_dict, inplace=True)
beach_locs = beach_locs.groupby('Beach').mean().reset_index()

### Forecast.io data

Daily forecast data

In [ ]:
forecast = pd.read_csv('../data/ExternalData/forecastio_daily_weather.csv')
forecast.time = pd.to_datetime(forecast.time)
forecast.rename(columns={'time': 'Date', 'beach': 'Beach'}, inplace=True)
forecast.Beach.replace(cleannames_dict, inplace=True)
forecast = forecast[~forecast.duplicated(subset=['Date', 'Beach'])]

Hourly forecast data

In [ ]:
hr_forecast = pd.read_csv('../data/ExternalData/forecastio_hourly_weather.csv')
hr_forecast.time = pd.to_datetime(hr_forecast.time)
hr_forecast.rename(columns={'time': 'Date', 'beach': 'Beach'}, inplace=True)
hr_forecast.Beach.replace(cleannames_dict, inplace=True)
hr_forecast = hr_forecast[~hr_forecast.duplicated(subset=['Date', 'Beach'])]

## Models

### Current Random Forest model

#### Model-specific data prep

In [ ]:
rf_df = df.copy()

#Add columns for 1 through 7 previous day ecoli readings
prev_days = [get_prev_nday(rf_df, 'Escherichia.coli', i) for i in range(1, 8)]
for prev_df in prev_days:
    rf_df = pd.merge(rf_df, prev_df, on=['Date', 'Beach'])

#Get historical average across those 7 days
prev_cols = [col for col in rf_df.columns if 'prev' in col]
rf_df['historical_average_Escherichia.coli'] = rf_df[prev_cols].mean(axis=1)

#Add in historical averages for daily forecast of temperatureMin and temperatureMax
rf_forecast = forecast.copy()
rf_forecast = rf_forecast.sort_values(by=['Beach', 'Date'])
rf_forecast['historical_average_temperatureMin'] =\
         rf_forecast.groupby(['Beach', rf_forecast.Date.dt.year])['temperatureMin']\
                  .apply(lambda x: x.rolling(min_periods=1, center=False, window=3).mean())
        
rf_forecast['historical_average_temperatureMax'] =\
         rf_forecast.groupby(['Beach', rf_forecast.Date.dt.year])['temperatureMax']\
                  .apply(lambda x: x.rolling(min_periods=1, center=False, window=3).mean())
        
#Separate hourly data into multiple columns, add previous day columns as well
rf_hr = hr_forecast.copy()
col_list = ['Date', 'Beach']+[col for col in rf_hr.columns 
                              if col != 'Date' and col != 'Beach' and col != 'precipType']
rf_hr = rf_hr[col_list]
rf_hr = rf_hr.sort_values(by='Date').reset_index(drop=True)
rf_hr = rf_hr.groupby('Beach').apply(pd.Series.interpolate)

df_list = []
for time in range(24):
    subhr_df = rf_hr[rf_hr.Date.dt.hour == time]\
                    [col_list].reset_index(drop=True)
    prefix = 'hr' + str(time) + '_'
    subhr_df.columns = col_list[:2] + [prefix + col 
                                       for col in col_list[2:]]
    subhr_df.Date = pd.DatetimeIndex(subhr_df.Date).normalize()
    subhr_df[prefix + 'summary'] = pd.Categorical(subhr_df[prefix + 'summary']).codes
    subhr_df[prefix + 'icon'] = pd.Categorical(subhr_df[prefix + 'icon']).codes
    
    df_list.append(subhr_df)
    
multi_hr = df_list[0]
for hr_df in df_list[1:]:
    multi_hr = pd.merge(multi_hr, hr_df, on=['Date', 'Beach'])
    
target = multi_hr.columns.tolist()[2:]
multi_hr = pd.merge(multi_hr, get_prev_nday(multi_hr, target, 1), on=['Date', 'Beach'])

#put it all together
rf_df = pd.merge(rf_df, multi_hr, on=['Beach', 'Date'])
rf_df = pd.merge(rf_df, rf_forecast, on=['Beach', 'Date'])
rf_df = pd.merge(rf_df, beach_locs, on=['Beach'])

#### Misc other feature engineering

In [ ]:
navy_pier = 41.8916
rf_df['n_pier'] = rf_df.Latitude > navy_pier

#### Model

In [ ]:
predictors = ['1_prev_hr13_temperature', 'precipIntensityMax', '1_prev_hr8_temperature',
              'hr1_windBearing', 'cloudCover', '1_prev_hr11_temperature', 'windSpeed', 
              'hr3_windBearing', 'hr2_windBearing', 'hr2_cloudCover', '1_prev_hr15_temperature',
              'humidity', 'hr4_windBearing', 'hr3_windSpeed', '1_prev_hr21_temperature', 'hr4_cloudCover',
              'hr4_windSpeed', '1_prev_hr12_temperature', 'hr0_temperature', 'hr0_pressure', 'precipIntensity', 
              'hr2_windSpeed', 'sunriseTime', 'temperatureMin', 'hr1_windSpeed', 'hr0_cloudCover', 
              '1_prev_Escherichia.coli', '2_prev_Escherichia.coli', '3_prev_Escherichia.coli',
              '4_prev_Escherichia.coli', '5_prev_Escherichia.coli', '6_prev_Escherichia.coli',
              '7_prev_Escherichia.coli', 'n_pier', 'historical_average_Escherichia.coli'
             ]
model = RandomForestClassifier(n_estimators=2000, max_depth=6, n_jobs=-1, class_weight={0: 1.0, 1: 1/.15})
fits, model_df = run_model(rf_df, model, predictors)

In [ ]:
metrics = calc_metrics(model_df, predictors, fits, 
                       metrics=['roc_curve', 'roc_auc_score'])

plt.figure(figsize=(8,8))
plot_roc(metrics['roc_curve'], drek_df, metrics['roc_auc_score'])

In [ ]:
get_feat_df(fits, predictors)